In [1]:
import numpy as np
import os
from keras.preprocessing import image
import string
from keras.models import load_model

Using TensorFlow backend.


In [2]:
captcha_word = string.digits + string.ascii_letters
word_len = 5
word_class = len(captcha_word)

width = 200
height = 60


train_dir = 'D:/data/YZM/second_test'
train_label =  'D:/data/YZM/second_test_label.txt'

#生成字符索引
char_indices = dict((c, i) for i,c in enumerate(captcha_word))
indices_char = dict((i, c) for i,c in enumerate(captcha_word))


def captcha_to_vec(captcha):    
   
    vector = np.zeros(word_len * word_class)
    for i,ch in enumerate(captcha):
        if ch == ',':
            continue
        idex = i * word_class + char_indices[ch]
        vector[idex] = 1
    return vector

def vec_to_captcha(vec):
    text = []
    vec[vec < 0.5] = 0    
    char_pos = vec.nonzero()[0]
    for i, ch in enumerate(char_pos):
        text.append(captcha_word[ch % word_class])
    return ''.join(text)


In [3]:
image_list = []

for item in os.listdir(train_dir):
    image_list.append(item)

print(len(image_list))

300


In [4]:
X = np.zeros((len(image_list), height, width, 3), dtype = np.uint8)
y = np.zeros((len(image_list), word_len * word_class), dtype = np.uint8)

with open(train_label) as file_object:
    p = file_object.readlines()
for i,img in enumerate(image_list):
    img_path = train_dir + "/" + img
    raw_img = image.load_img(img_path, target_size=(height, width))
    X[i] = image.img_to_array(raw_img)
    for j in range(len(p)):
        if img[:4]==p[j][:4]:
            break;
    y[i] = captcha_to_vec(p[j][5:10])


In [5]:
model=load_model('second_model.h5')    

def testCaptcha(index):
    raw_img = X[index]
    true_label = y[index]
    
    X_T = np.zeros((1, height, width, 3), dtype = np.float32)
    X_T[0] = image.img_to_array(raw_img)
    
    
    result = model.predict(X_T)
    
    vex_test = vec_to_captcha(result[0])
    true_test = vec_to_captcha(true_label)
    
    
    if vex_test ==true_test:
        return 1
    else:
        return 0
    
    
c=0
for i in range(300):
    if testCaptcha(i)==1:
               c+=1
print(c/300)

0.97
